In [38]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import calendar
from scipy.stats import chi2_contingency
from IPython.display import display
import os, re
project_folder =os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))

In [3]:
data_general = pd.read_csv(f'{project_folder}/data/process/final_geo_coding.csv', skiprows=[1, 2]).reset_index()

In [5]:
# show the questions
pd.read_csv(f'{project_folder}/data/process/new_data.csv', skiprows=[2]).reset_index().iloc[0]

index                                                          0
StartDate                                             Start Date
EndDate                                                 End Date
Status                                             Response Type
IPAddress                                             IP Address
                                     ...                        
Q38            What is the highest level of education you hav...
Q39            What do you consider your main ethnic origin o...
Q39_10_TEXT    What do you consider your main ethnic origin o...
Q40            Where did you hear about this survey? - Select...
Q40_3_TEXT     Where did you hear about this survey? - Other ...
Name: 0, Length: 75, dtype: object

In [35]:
questions ={'Q2':'part','Q6':'day_time','Q7':'is_locked','Q8':'locked_type','Q9':'theft_location','Q10':'is_regi','Q11':'report_loc','Q12':'is_insured','Q13':'estimate_val','Q14':'is_electric','Q15':'bike_type','Q17':'is_university','Q19':'is_online','Q21':'is_police_assist','Q23':'recovery_loc','Q24':'con_recovery','Q34': 'birth', 'Q35': 'gender', 'Q36': 'income', 'Q38': 'education', 'Q39': 'ethnic_origin'}

In [102]:
percentage = 'percentage'
for pair in questions.items():

    # statistic by count (display also by percentage)
    question = pair[0]
    # later code for birth year
    if question =='Q34':
        continue
    table = DataFrame(data_general.groupby(question).count()['level_0'].sort_values(ascending=False).rename('count'))
    table[percentage] = round(table['count']/table['count'].sum()*100,1)
    # Aggregate responses type with less than 10 responses
    table_aggregate = table[table[percentage]<10]
    table_new = table.copy()
    if len(table_aggregate)>1:
        table_new =table[table[percentage]>10]
        if table_aggregate[percentage].sum()<10:
            table_new.loc['Aggregated Responses'] = round(table_aggregate.sum(),1)
        else:
            # If the aggregate percentage is more than 10 take some out
            table_new = table_new.append(table_aggregate.iloc[0])
            table_new.loc['Aggregated Responses'] = round(table_aggregate.iloc[1:].sum(),1)

    table_new[percentage] = table_new[percentage].astype(str) + '%'
    print(f'{pair[1]}:{table_new}\n')


part:                                                                                      count  \
Q2                                                                                            
Entire bicycle (i.e., frame and components)                                            1782   
A major bicycle part necessary for riding (i.e., saddle, wheel, pedal, or handlebar)     24   

                                                                                     percentage  
Q2                                                                                               
Entire bicycle (i.e., frame and components)                                               98.7%  
A major bicycle part necessary for riding (i.e., saddle, wheel, pedal, or handlebar)       1.3%  

day_time:                         count percentage
Q6                                       
Overnight (00:00-05:59)    734      40.5%
Afternoon (12:00-17:59)    356      19.6%
Don’t know/not sure        328      18.1%
Ev

C:\Users\Achituv\AppData\Local\Temp\ipykernel_19912\2199857159.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_new.loc['Aggregated Responses'] = round(table_aggregate.sum(),1)
C:\Users\Achituv\AppData\Local\Temp\ipykernel_19912\2199857159.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_new[percentage] = table_new[percentage].astype(str) + '%'
C:\Users\Achituv\AppData\Local\Temp\ipykernel_19912\2199857159.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

In [112]:
def get_the_highest(table_updated, table_to_agg):
    if table_to_agg[percentage].sum()<10:
        table_updated.loc['Aggregated Responses'] = round(table_to_agg.sum(), 1)
        return table_updated
    else:
        table_updated = table_updated.append(table_to_agg.iloc[0])
        return get_the_highest(table_updated, table_to_agg.iloc[1:])


In [123]:
def sub_aggregate_def(indices:list,j,sum):
    temp_sum = sub_aggregate.iloc[j][percentage] +sum
    if sum>10:
        return indices
    else:
        indices.append(j)
        sub_aggregate_def(indices,j+1,temp_sum)

In [124]:
pair = ('Q8','estimate_val')
percentage = 'percentage'
question = pair[0]

# later code for birth year

table = DataFrame(data_general.groupby(question).count()['level_0'].sort_values(ascending=False).rename('count'))

table[percentage] = round(table['count']/table['count'].sum()*100,1)
# Aggregate responses type with less than 10 responses
table_aggregate = table[table[percentage]<10]
table_new = table.copy()
if len(table_aggregate)>1:
    table_new =table[table[percentage]>10]
    table_add = get_the_highest(DataFrame(),table_aggregate)
    if len(table_add)>2:
        sub_aggregate = table_add.iloc[:-1].sort_values(by=percentage)
        i =0
        while i<len(sub_aggregate):
            indices = sub_aggregate_def([i],i+1,sub_aggregate.iloc[i][percentage])
            table_new.append(sub_aggregate.iloc[indices])
            i = indices[-1]+1
        table_new.append(table_add.iloc[-1])
    else:
        table_new = table_new.append(table_add)
    # if table_aggregate[percentage].sum()<10:
    #     table_new.loc['Aggregated Responses'] = round(table_aggregate.sum(),1)
    # else:
    #     table_new = table_new.append(table_aggregate.iloc[0])
    #     table_new.loc['Aggregated Responses'] = round(table_aggregate.iloc[1:].sum(),1)



table_new[percentage] = table_new[percentage].astype(str) + '%'
table_new



C:\Users\Achituv\AppData\Local\Temp\ipykernel_19912\3194428247.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_updated = table_updated.append(table_to_agg.iloc[0])
C:\Users\Achituv\AppData\Local\Temp\ipykernel_19912\3194428247.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_updated = table_updated.append(table_to_agg.iloc[0])
C:\Users\Achituv\AppData\Local\Temp\ipykernel_19912\3194428247.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_updated = table_updated.append(table_to_agg.iloc[0])
C:\Users\Achituv\AppData\Local\Temp\ipykernel_19912\3194428247.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_u

TypeError: Cannot index by location index with a non-integer key

In [134]:
sub_aggregate.iloc[[1,2]]

,count,percentage
Two U-Locks,14.0,1.3
"Cable lock ,Other (please specify):",16.0,1.5


In [135]:
table_aggregate

,count,percentage
Q8,,
Chain-lock combination (chain with mini u-lock or integrated locking mechanism),83,7.8
Other (please specify):,71,6.7
"​U-lock ,Cable lock",50,4.7
Chain-lock with padlock,47,4.4
Folding lock,24,2.2
"​U-lock ,Chain-lock combination (chain with mini u-lock or integrated locking mechanism)",17,1.6
"Cable lock ,Other (please specify):",16,1.5
Two U-Locks,14,1.3
"​U-lock ,Locking skewers",11,1.0
